
# 전국 신규 민간 아파트 분양가격 동향

2013년부터 최근까지 부동산 가격 변동 추세가 아파트 분양가에도 반영될까요? 공공데이터 포털에 있는 데이터를 Pandas 의 melt, concat, pivot, transpose 와 같은 reshape 기능을 활용해 분석해 봅니다. 그리고 groupby, pivot_table, info, describe, value_counts 등을 통한 데이터 요약과 분석을 해봅니다. 이를 통해 전혀 다른 형태의 두 데이터를 가져와 정제하고 병합하는 과정을 다루는 방법을 알게 됩니다. 전처리 한 결과에 대해 수치형, 범주형 데이터의 차이를 이해하고 다양한 그래프로 시각화를 할 수 있게 됩니다.


## 다루는 내용

* 공공데이터를 활용해 전혀 다른 두 개의 데이터를 가져와서 전처리 하고 병합하기
* 수치형 데이터와 범주형 데이터를 바라보는 시각을 기르기
* 데이터의 형식에 따른 다양한 시각화 방법 이해하기

## 실습
* 공공데이터 다운로드 후 주피터 노트북으로 로드하기
* 판다스를 통해 데이터를 요약하고 분석하기
* 데이터 전처리와 병합하기
* 수치형 데이터와 범주형 데이터 다루기
* 막대그래프(bar plot), 선그래프(line plot), 산포도(scatter plot), 상관관계(lm plot), 히트맵, 상자수염그림, swarm plot, 도수분포표, 히스토그램(distplot) 실습하기

## 데이터 출처
* https://kosis.kr/statHtml/statHtml.do?orgId=414&tblId=DT_41401N_005&vw_cd=MT_ZTITLE&list_id=I1_3&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE

In [1]:
# !pip install koreanize_matplotlib

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data/apt-price-hug.csv", encoding="cp949")
df.shape

(10190, 5)

In [4]:
df.head()

,시점,규모별(1),지역별(1),지역별(2),데이터
0,2015.1,전체,수도권,소계,4047
1,2015.1,전체,수도권,서울,5841
2,2015.1,전체,수도권,인천,3163
3,2015.1,전체,수도권,경기,3138
4,2015.1,전체,5대광역시 및 세종특별자치시,소계,2727


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10190 entries, 0 to 10189
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시점      10190 non-null  float64
 1   규모별(1)  10190 non-null  object 
 2   지역별(1)  10190 non-null  object 
 3   지역별(2)  10190 non-null  object 
 4   데이터     10190 non-null  object 
dtypes: float64(1), object(4)
memory usage: 398.2+ KB


In [6]:
df.isnull().sum()

시점        0
규모별(1)    0
지역별(1)    0
지역별(2)    0
데이터       0
dtype: int64

In [7]:
df.columns

Index(['시점', '규모별(1)', '지역별(1)', '지역별(2)', '데이터'], dtype='object')

In [8]:
df.columns = ['시점', '전용면적', '지역구분', '시도', '분양가']
df

,시점,전용면적,지역구분,시도,분양가
0,2015.10,전체,수도권,소계,4047
1,2015.10,전체,수도권,서울,5841
2,2015.10,전체,수도권,인천,3163
3,2015.10,전체,수도권,경기,3138
4,2015.10,전체,5대광역시 및 세종특별자치시,소계,2727
...,...,...,...,...,...
10185,2024.03,전용면적 102㎡초과,기타지방,전북,4056
10186,2024.03,전용면적 102㎡초과,기타지방,전남,4572
10187,2024.03,전용면적 102㎡초과,기타지방,경북,4607
10188,2024.03,전용면적 102㎡초과,기타지방,경남,4396


In [9]:
df = df[df["시도"] != "소계"].copy()
df.shape

(8660, 5)

In [10]:
df["시도"].value_counts()

시도
세종    510
전북    510
경기    510
부산    510
대구    510
경남    510
경북    510
인천    510
강원    510
충북    510
충남    510
전남    509
서울    509
울산    509
제주    509
광주    508
대전    506
Name: count, dtype: int64

In [11]:
df["지역구분"]

1                    수도권
2                    수도권
3                    수도권
5        5대광역시 및 세종특별자치시
6        5대광역시 및 세종특별자치시
              ...       
10185               기타지방
10186               기타지방
10187               기타지방
10188               기타지방
10189               기타지방
Name: 지역구분, Length: 8660, dtype: object

In [12]:
df.describe()

,시점
count,8660.000000
mean,2019.562905
std,2.472554
min,2015.100000
25%,2017.110000
50%,2019.120000
75%,2022.020000
max,2024.030000


In [13]:
df["분양가천원"] = pd.to_numeric(df["분양가"], errors="coerce")
df["분양가천원"]

1        5841.0
2        3163.0
3        3138.0
5        3112.0
6        2682.0
          ...  
10185    4056.0
10186    4572.0
10187    4607.0
10188    4396.0
10189    7880.0
Name: 분양가천원, Length: 8660, dtype: float64

In [14]:
df.describe()

,시점,분양가천원
count,8660.000000,7991.000000
mean,2019.562905,3864.790890
std,2.472554,1648.643403
min,2015.100000,1868.000000
25%,2017.110000,2714.500000
50%,2019.120000,3396.000000
75%,2022.020000,4439.000000
max,2024.030000,14918.000000


In [15]:
df.describe(include="object")

,전용면적,지역구분,시도,분양가
count,8660,8660,8660,8660
unique,5,3,17,2998
top,전체,기타지방,세종,-
freq,1734,4078,510,669


In [16]:
df["연도"] = df["시점"].astype(str).str[:4].astype(int)
df["월"] = df["시점"].map(lambda x : str(x).split(".")[-1]).astype(int)
df

,시점,전용면적,지역구분,시도,분양가,분양가천원,연도,월
1,2015.10,전체,수도권,서울,5841,5841.0,2015,1
2,2015.10,전체,수도권,인천,3163,3163.0,2015,1
3,2015.10,전체,수도권,경기,3138,3138.0,2015,1
5,2015.10,전체,5대광역시 및 세종특별자치시,부산,3112,3112.0,2015,1
6,2015.10,전체,5대광역시 및 세종특별자치시,대구,2682,2682.0,2015,1
...,...,...,...,...,...,...,...,...
10185,2024.03,전용면적 102㎡초과,기타지방,전북,4056,4056.0,2024,3
10186,2024.03,전용면적 102㎡초과,기타지방,전남,4572,4572.0,2024,3
10187,2024.03,전용면적 102㎡초과,기타지방,경북,4607,4607.0,2024,3
10188,2024.03,전용면적 102㎡초과,기타지방,경남,4396,4396.0,2024,3


In [17]:
df.describe()

,시점,분양가천원,연도,월
count,8660.000000,7991.000000,8660.000000,8660.000000
mean,2019.562905,3864.790890,2019.497921,5.710624
std,2.472554,1648.643403,2.477126,3.660636
min,2015.100000,1868.000000,2015.000000,1.000000
25%,2017.110000,2714.500000,2017.000000,2.000000
50%,2019.120000,3396.000000,2019.000000,5.000000
75%,2022.020000,4439.000000,2022.000000,9.000000
max,2024.030000,14918.000000,2024.000000,12.000000
